In [159]:
from deep_translator import DeeplTranslator, PapagoTranslator, GoogleTranslator
import pandas as pd
import re
import time
import random

In [136]:
from huggingface_hub import login
import importlib
import token_login
importlib.reload(token_login)
login(token= token_login.kl_token)

In [ ]:
from datasets import load_dataset

ds = load_dataset("msarmi9/korean-english-multitarget-ted-talks-task")

C:\Users\krist\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\krist\.cache\huggingface\hub\datasets--msarmi9--korean-english-multitarget-ted-talks-task. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(messa

In [25]:
ds

DatasetDict({
    train: Dataset({
        features: ['korean', 'english'],
        num_rows: 166215
    })
    validation: Dataset({
        features: ['korean', 'english'],
        num_rows: 1958
    })
    test: Dataset({
        features: ['korean', 'english'],
        num_rows: 1982
    })
})

In [26]:
# If 'dataset' is a DatasetDict, select a split first
if isinstance(ds, dict): # Check if it's a DatasetDict
    train_df = ds['train'].to_pandas() # Example: convert the 'train' split
    test_df = ds['test'].to_pandas()
    val_df = ds['validation'].to_pandas()
else: # If 'dataset' is already a Dataset object
    df = ds.to_pandas()

In [28]:
test_df.head()

,korean,english
0,아직 3분 시작된 건 아니죠? 그렇죠?,Allison Hunt: My three minutes hasn't started ...
1,"크리스 앤더슨:네, 맘대로 시작하실 수 없습니다.","Chris Anderson: No, you can't start the three ..."
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair."
3,"앨리슨 헌트 : 어머나, 여기 참 냉정하네요","AH: Oh my God, it's harsh up here."
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.


In [33]:
df

,korean,english
0,"(박수) 이쪽은 Bill Lange 이고, 저는 David Gallo입니다",(Applause) David Gallo: This is Bill Lange. I'...
1,우리는 여러분에게 바닷속 이야기를 영상과 함께 들려주고자 합니다.,And we're going to tell you some stories from ...
2,저희는 끝내주는 타이타닉 비디오도 있긴 합니다만 뭐..여기서는 눈꼽만큼도 보여줄 생...,We've got some of the most incredible video of...
3,(웃음) 비록 타이타닉이 박스오피스에서 굉장한 실적을 거두긴 했지만 바다가 들려주는...,(Laughter) The truth of the matter is that the...
4,문제라면 우리는 우리가 바다를 이미 알고있다고 믿는거죠.,"And the problem, I think, is that we take the ..."
...,...,...
166210,제 셔츠의 입자로 만들었지요.,And that was done with a particle from my shirt.
166211,그리고 이 못 머리의 초록색은 초록색 셔츠의 입자를 긁어내어 못 머리에 눌러주었습니다.,And the pinhead I've made green around there b...
166212,제 작품 활동은 꽤나 공이 들어가는 일입니다. 하지만 최고의 것들은 항상 작은 것에...,"It's very painstaking work, but the best thing..."
166213,(웃음) 부르노 지우사니: 윌라드 위간 씨였습니다!,(Laughter) Bruno Giussani: Willard Wigan!


In [34]:
total_characters = sum(len(s) for s in df['korean'])
print(total_characters)

8438962


In [38]:
langs_list = GoogleTranslator().get_supported_languages()  # output: [arabic, french, english etc...]
print(langs_list)

['afrikaans', 'albanian', 'amharic', 'arabic', 'armenian', 'assamese', 'aymara', 'azerbaijani', 'bambara', 'basque', 'belarusian', 'bengali', 'bhojpuri', 'bosnian', 'bulgarian', 'catalan', 'cebuano', 'chichewa', 'chinese (simplified)', 'chinese (traditional)', 'corsican', 'croatian', 'czech', 'danish', 'dhivehi', 'dogri', 'dutch', 'english', 'esperanto', 'estonian', 'ewe', 'filipino', 'finnish', 'french', 'frisian', 'galician', 'georgian', 'german', 'greek', 'guarani', 'gujarati', 'haitian creole', 'hausa', 'hawaiian', 'hebrew', 'hindi', 'hmong', 'hungarian', 'icelandic', 'igbo', 'ilocano', 'indonesian', 'irish', 'italian', 'japanese', 'javanese', 'kannada', 'kazakh', 'khmer', 'kinyarwanda', 'konkani', 'korean', 'krio', 'kurdish (kurmanji)', 'kurdish (sorani)', 'kyrgyz', 'lao', 'latin', 'latvian', 'lingala', 'lithuanian', 'luganda', 'luxembourgish', 'macedonian', 'maithili', 'malagasy', 'malay', 'malayalam', 'maltese', 'maori', 'marathi', 'meiteilon (manipuri)', 'mizo', 'mongolian', 'm

In [92]:
# remove speaker identification (e.g. "Bill Lange: ...")
# remove aural indicators
#r'[\uAC00-\uD7A3]+\s?[\uAC00-\uD7A3]+:\s', r'\([\uAC00-\uD7A3]+\)\s?'
pattern = [r'\w+(\s\w+)?\:\s', r'\(\w+\)\s?', r'[\\p{L}]+(\s[\\p{L}]+)?:\s', r'\([\\p{L}]+\)\s?']
remove_str = "|".join(pattern)

train_df['clean english'] = train_df['english'].str.replace(remove_str, "",regex=True)
train_df['clean korean'] = train_df['korean'].str.replace(remove_str, "", regex=True)

test_df['clean english'] = test_df['english'].str.replace(remove_str, "",regex=True)
test_df['clean korean'] = test_df['korean'].str.replace(remove_str, "", regex=True)

val_df['clean english'] = val_df['english'].str.replace(remove_str, "",regex=True)
val_df['clean korean'] = val_df['korean'].str.replace(remove_str, "", regex=True)

In [ ]:
test_df.tail()

,korean,english,clean english,clean korean
1977,W: 예. 에너지와 관련한 일할 생각을 갖고 있습니다.,WK: Yeah. I'm still thinking to work on energy.,Yeah. I'm still thinking to work on energy.,예. 에너지와 관련한 일할 생각을 갖고 있습니다.
1978,"C: 윌리엄, TED에 당신을 초대하게 되어 영광입니다.","CA: Wow. William, it's a real honor to have yo...","Wow. William, it's a real honor to have you at...","윌리엄, TED에 당신을 초대하게 되어 영광입니다."
1979,와줘서 정말 고마워요.,Thank you so much for coming.,Thank you so much for coming.,와줘서 정말 고마워요.
1980,W: 감사합니다.,WK: Thank you.,Thank you.,감사합니다.
1981,(박수),(Applause),,


In [97]:
# remove empty rows
clean_train_df = train_df[train_df['clean english']!= '']
clean_test_df = test_df[test_df['clean english']!= '']
clean_val_df = val_df[val_df['clean english']!= '']
print(len(val_df))
print(len(clean_val_df))
clean_val_df.tail()

1958
1957


,korean,english,clean english,clean korean
1952,"저희 회사는 사진 7억장 보유하고 있지만,",My company has 70 million images.,My company has 70 million images.,"저희 회사는 사진 7억장 보유하고 있지만,"
1953,제 사무실에는 한 장의 사진이 있습니다.,I have one image in my office.,I have one image in my office.,제 사무실에는 한 장의 사진이 있습니다.
1954,이 사진이죠.,Here it is.,Here it is.,이 사진이죠.
1955,전 다음번에는 여러분이 여러분을 행동하게 할 사진을 만나길 바랍니다. 사진으로 이유...,I hope that the next time you see an image tha...,I hope that the next time you see an image tha...,전 다음번에는 여러분이 여러분을 행동하게 할 사진을 만나길 바랍니다. 사진으로 이유...
1956,모든 사진가들에게 감사를 드립니다.,And thank you to all the photographers.,And thank you to all the photographers.,모든 사진가들에게 감사를 드립니다.


In [173]:
result_df = pd.DataFrame(columns=['korean', 'actual translation', 'Google translation', 'DeepL translation'])

# Convert the Korean column to a list
korean_texts = clean_test_df['clean korean'].tolist()

# Batch size for GoogleTranslator (safe max = 40)
BATCH_SIZE = 30

translator = GoogleTranslator(source='ko', target='en')

google_translations = []

# Process in batches
for i in range(0, len(korean_texts), BATCH_SIZE):
    batch = korean_texts[i:i + BATCH_SIZE]
    print("batch: ", i)

    for attempt in range(3):
        try:
            translated_batch = translator.translate_batch(batch)
            google_translations.extend(translated_batch)
            break
        except Exception as e:
            print(f"Batch failed (attempt {attempt+1}): {e}")
            sleep_time = 2 ** attempt + random.random()  # exponential backoff
            print(f"Sleeping {sleep_time:.2f} sec...")
            time.sleep(sleep_time)

    time.sleep(0.3) 

# DeepL translation will stay empty unless you use DeepL Pro
# result_df['DeepL translation'] = ""

# for txt in range(0, len(clean_train_df['clean korean'])):
#     translated = GoogleTranslator(source='auto', target='en').translate(text=clean_train_df['clean korean'][txt]) 
    
#     new_row_df = pd.DataFrame({'korean': [clean_train_df['clean korean'][txt]], 'actual translation':[clean_train_df['clean english'][txt]], 'Google translation':[translated]})
#     result_df = pd.concat([result_df, new_row_df], ignore_index=True)

batch:  0
Batch failed (attempt 1): ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Sleeping 1.11 sec...
batch:  30
batch:  60
batch:  90
batch:  120
batch:  150
batch:  180
batch:  210
batch:  240
batch:  270
batch:  300
batch:  330
batch:  360
batch:  390
batch:  420
batch:  450
batch:  480
batch:  510
batch:  540
batch:  570
batch:  600
batch:  630
batch:  660
batch:  690
batch:  720
batch:  750
batch:  780
batch:  810
batch:  840
batch:  870
batch:  900
batch:  930
batch:  960
batch:  990
batch:  1020
batch:  1050
batch:  1080
Batch failed (attempt 1): ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Sleeping 1.98 sec...
batch:  1110
batch:  1140
batch:  1170
batch:  1200
batch:  1230
batch:  1260
batch:  1290
batch:  1320
batch:  1350
batch:  1380
batch:  1410
batch:  1440
batch:  1470
batch:  1500
batch:  

In [174]:
# Assemble the final DataFrame
result_df['korean'] = clean_test_df['clean korean']
result_df['actual translation'] = clean_test_df['clean english']
result_df['Google translation'] = google_translations

In [177]:
result_df.to_csv('data/korean_translation.csv', index=False)


In [145]:
deepl_translator = DeeplTranslator(api_key=token_login.deepl_key, use_free_api=True)

In [153]:
print(dir(deepl_translator))


['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_base_url', '_element_query', '_element_tag', '_languages', '_map_language_to_code', '_read_docx', '_read_pdf', '_same_source_target', '_source', '_supported_languages', '_target', '_translate_batch', '_translate_file', '_type', '_url_params', 'api_key', 'get_supported_languages', 'is_language_supported', 'payload_key', 'source', 'target', 'translate', 'translate_batch', 'translate_file', 'version']


In [152]:
translated = deepl_translator.translate_text(clean_train_df['clean korean'][0], source = 'ko', target = 'en')

AttributeError: 'DeeplTranslator' object has no attribute 'translate_text'

In [151]:
print(translated)
print(clean_train_df['clean korean'][0])

이쪽은 Bill Lange 이고, 저는 David Gallo입니다
이쪽은 Bill Lange 이고, 저는 David Gallo입니다


In [139]:
# using DeepL translator 500,000 character limit/month
translated = DeeplTranslator(token_login.deepl_key).translate_batch(clean_train_df['clean korean'][0:5])

ApiKeyException: None --> 
You have to pass your api_key!
You can do this by passing the key as a parameter/argument to the translator class
or by setting the environment variable DEEPL_API_KEY

Example: export DEEPL_API_KEY="your_api_key"


In [176]:
pd.set_option('display.max_colwidth', None)

result_df.head()

,korean,actual translation,Google translation,DeepL translation
0,아직 3분 시작된 건 아니죠? 그렇죠?,"My three minutes hasn't started yet, has it?","The 3 minutes haven't started yet, right? Right?",NaN
1,"크리스 앤더슨:네, 맘대로 시작하실 수 없습니다.","No, you can't start the three minutes.","Chris Anderson: Yes, you can't just start.",NaN
2,"3분 다시 설정해주세요, 이건 반칙입니다.","Reset the three minutes, that's just not fair.","Please reset the time to 3 minutes, this is a foul.",NaN
3,"앨리슨 헌트 : 어머나, 여기 참 냉정하네요","Oh my God, it's harsh up here.","Allison Hunt: Oh my, it's so cold in here.",NaN
4,정말이지 긴장되네요,I mean I'm nervous enough as it is.,I'm really nervous.,NaN
